In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np
import pandas as pd
from kaggle.competitions import twosigmanews

env = twosigmanews.make_env()
(marketdf, newsdf) = env.get_training_data()

from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold

import lightgbm as lgb
import matplotlib.pyplot as plt
import seaborn as sns
import pickle

In [ ]:
def prepare_data(marketdf, newsdf):
    a bit of feature engineering
    marketdf['time'] = marketdf.time.dt.strftime("%Y%m%d").astype(int)
    marketdf['bartrend'] = marketdf['close'] / marketdf['open']
    marketdf['average'] = (marketdf['close'] + marketdf['open'])/2
    marketdf['pricevolume'] = marketdf['volume'] * marketdf['close']
    
    newsdf['time'] = newsdf.time.dt.strftime("%Y%m%d").astype(int)
    newsdf['assetCode'] = newsdf['assetCodes'].map(lambda x: list(eval(x))[0])
    newsdf['position'] = newsdf['firstMentionSentence'] / newsdf['sentenceCount']
    newsdf['coverage'] = newsdf['sentimentWordCount'] / newsdf['wordCount']

    # filter pre-2012 data, no particular reason
    marketdf = marketdf.loc[marketdf['time'] > 20120000]
    
    # get rid of extra junk from news data
    try:
        droplist = ['sourceTimestamp','firstCreated','sourceId','headline','takeSequence','provider','firstMentionSentence',
                    'sentenceCount','bodySize','headlineTag','marketCommentary','subjects','audiences','sentimentClass',
                    'assetName', 'assetCodes','urgency','wordCount','sentimentWordCount']
        newsdf.drop(droplist, axis=1, inplace=True)
    except:
        print('droplist has been dropped in newsdf')
        
    marketdf.drop(['assetName', 'volume'], axis=1, inplace=True)
    
    # combine multiple news reports for same assets on same day
    newsgp = newsdf.groupby(['time','assetCode'], sort=False).aggregate(np.mean).reset_index()
    
    # join news reports to market data, note many assets will have many days without news data
    return pd.merge(marketdf, newsgp, how='left', on=['time', 'assetCode'], copy=False) #, right_on=['time', 'assetCodes'])


def post_scaling(df):
    mean, std = np.mean(df), np.std(df)
    df = (df - mean)/ (std * 2)     # original: (df - mean)/ (std * 8) 
    return np.clip(df,-1,1)

def post_scaling_nonVarScaling(df):
    mean, std = np.mean(df), np.std(df)
    df = (df - mean)/ (std)
    return np.clip(df,-1,1)




# read and save all test data

In [ ]:
import time
print('=========> read all test data')
market_obs_df = None
news_obs_df = None
pred_df = None
every_days = env.get_prediction_days()
news_droplist = ['sourceTimestamp','firstCreated','sourceId','headline','takeSequence','provider','firstMentionSentence',
                'sentenceCount','bodySize','headlineTag','marketCommentary','subjects','audiences','sentimentClass',
                'assetName', 'assetCodes','urgency','wordCount','sentimentWordCount']
load_time = 0
for (m_df, n_df, predictions_template_df) in every_days:
    t_start = time.clock()
    env.predict(predictions_template_df)
    predictions_template_df['time'] = m_df.time.min()
    if market_obs_df is None:
        market_obs_df = m_df
        news_obs_df = n_df.drop(news_droplist, axis=1)
        pred_df = predictions_template_df
    else:
        market_obs_df = market_obs_df.append(m_df, ignore_index=True)
        news_obs_df = news_obs_df.append(n_df.drop(news_droplist, axis=1), ignore_index=True)
        pred_df = pred_df.append(predictions_template_df, ignore_index=True)
    t_end = time.clock()
    load_time += (t_end-t_start)
    print('single turn time:',t_end-t_start,'total time:',load_time)
### save test data （为了防止后面的操作给写乱了）
print('=========> save test data to txt file')
fw = open('market_obs_df.txt','wb')
pickle.dump(market_obs_df, fw, -1)  
fw.close()
fw = open('news_obs_df.txt','wb')
pickle.dump(news_obs_df, fw, -1)  
fw.close()
fw = open('pred_df.txt','wb')
pickle.dump(pred_df, fw, -1)  
fw.close()

# build trainset using train data

In [ ]:
print('============> preparing data...')
cdf = prepare_data(marketdf, newsdf)    
del marketdf, newsdf  # save the precious memory

print('============> building training set...')
targetcols = ['returnsOpenNextMktres10']
traincols = [col for col in cdf.columns if col not in ['time', 'assetCode', 'universe'] + targetcols]


In [ ]:
### we be classifyin
# left_threshold = -1
# right_threshold = 1
# ## 直接把超出left_threshold和right_threshold的数据扔掉
# trainset_drop_index_left = cdf[cdf[targetcols[0]] < left_threshold].index
# cdf.drop(trainset_drop_index_left, axis='index', inplace=True)
# cdf.reset_index(drop=True)
# trainset_drop_index_right = cdf[cdf[targetcols[0]] > right_threshold].index
# cdf.drop(trainset_drop_index_right, axis='index', inplace=True)
# cdf.reset_index(drop=True)
## 采用clip方案
# cdf[targetcols[0]].clip(left_threshold,right_threshold, inplace=True)

### 作者原始的做法
cdf[targetcols[0]] = (cdf[targetcols[0]] > 0).astype(int)     # kaggle public kernel的数据截断方法


### split train set and valid set
dates = cdf['time'].unique()
train = range(len(dates))[:int(0.85*len(dates))]
val = range(len(dates))[int(0.85*len(dates)):]

### train data
Xt = cdf[traincols].fillna(0).loc[cdf['time'].isin(dates[train])].values
Yt = cdf[targetcols].fillna(0).loc[cdf['time'].isin(dates[train])].values

### validation data
Xv = cdf[traincols].fillna(0).loc[cdf['time'].isin(dates[val])].values
Yv = cdf[targetcols].fillna(0).loc[cdf['time'].isin(dates[val])].values

print(Xt.shape, Xv.shape)


In [ ]:
print ('============> Training lightgbm')
########## params for lgb (作者原始的参数)
# params = {"objective" : "binary",
#           "metric" : "binary_logloss",
#           "num_leaves" : 125, # originally 60
#           "max_depth": -1,
#           "learning_rate" : 0.0005,   # originally .01
#           "bagging_fraction" : 0.9,  # subsample
#           "feature_fraction" : 0.9,  # colsample_bytree
#           "bagging_freq" : 5,        # subsample_freq
#           "bagging_seed" : 2018,
#           "verbosity" : -1 }

########## params for lgb 
params = {"objective" : "binary",
          "metric" : "binary_logloss",
        #   "lambda_l1": 0.01,
          "num_leaves" : 255, # originally 60
          "max_depth": -1,
          "learning_rate" : 0.0005,   # originally .01
          "bagging_fraction" : 0.9,  # subsample
          "feature_fraction" : 0.9,  # colsample_bytree
          "bagging_freq" : 5,        # subsample_freq
          "bagging_seed" : 2018,
          "verbosity" : -1 }


## We can introduce other boosting algos, default is traditional gradient boosting decision tree
## Other options include random forest (rf), dropouts meet multiple additive regression trees (dart), 
## or gradient-based on one-side sampling 

lgtrain, lgval = lgb.Dataset(Xt, Yt[:,0]), lgb.Dataset(Xv, Yv[:,0])
lgbmodel = lgb.train(params, lgtrain, 2600, valid_sets=[lgtrain, lgval], early_stopping_rounds=300, verbose_eval=200)




# retrain model using testset data

In [ ]:
def prepare_data_from_testset(marketdf, newsdf):
    # a bit of feature engineering
    marketdf['time'] = marketdf.time.dt.strftime("%Y%m%d").astype(int)
    marketdf['bartrend'] = marketdf['close'] / marketdf['open']
    marketdf['average'] = (marketdf['close'] + marketdf['open'])/2
    marketdf['pricevolume'] = marketdf['volume'] * marketdf['close']
    
    # test集合上的returnsOpenNextMktres10没有用，直接扔掉
    marketdf.drop(['returnsOpenNextMktres10'],axis=1)
    marketdf['new_returnsOpenNextMktres10'] = marketdf.groupby(['assetCode'])['returnsOpenPrevMktres10'].shift(-11).fillna(0)
    
    newsdf['time'] = newsdf.time.dt.strftime("%Y%m%d").astype(int)
    newsdf['assetCode'] = newsdf['assetCodes'].map(lambda x: list(eval(x))[0])
    newsdf['position'] = newsdf['firstMentionSentence'] / newsdf['sentenceCount']
    newsdf['coverage'] = newsdf['sentimentWordCount'] / newsdf['wordCount']

    # filter pre-2012 data, no particular reason
    marketdf = marketdf.loc[marketdf['time'] > 20120000]
    # get rid of extra junk from news data
    try:
        droplist = ['sourceTimestamp','firstCreated','sourceId','headline','takeSequence','provider','firstMentionSentence',
                    'sentenceCount','bodySize','headlineTag','marketCommentary','subjects','audiences','sentimentClass',
                    'assetName', 'assetCodes','urgency','wordCount','sentimentWordCount']
        newsdf.drop(droplist, axis=1, inplace=True)
    except:
        print('droplist has been dropped in newsdf')
        
    marketdf.drop(['assetName', 'volume'], axis=1, inplace=True)
    # combine multiple news reports for same assets on same day
    newsgp = newsdf.groupby(['time','assetCode'], sort=False).aggregate(np.mean).reset_index()
    # join news reports to market data, note many assets will have many days without news data
    return pd.merge(marketdf, newsgp, how='left', on=['time', 'assetCode'], copy=False) #, right_on=['time', 'assetCodes'])



print("============> training model with test_set data...")
try:
    del cdf
except:
    print('rerun! cdf has been deleted')
cdf = prepare_data_from_testset(market_obs_df, news_obs_df)

try:
    del market_obs_df, news_obs_df, pred_df
except:
    print('rerun! market_obs_df, news_obs_df, pred_df have been deleted')

try:
    del lgtrain, lgval
except:
    print('rerun! lgtrain, lgval have been deleted')    

print("---------------------------> building train data from test set...")
targetcols = ['new_returnsOpenNextMktres10'] # 将这一列直接作为y
traincols = [col for col in cdf.columns if col not in ['time', 'assetCode', 'universe'] + targetcols]
### 作者原始的做法
cdf[targetcols[0]] = (cdf[targetcols[0]] > 0).astype(int)     # kaggle public kernel的数据截断方法
### split train set and valid set
dates = cdf['time'].unique()
train = range(len(dates))[:int(0.90*len(dates))]
val = range(len(dates))[int(0.90*len(dates)):]
### train data
Xt = cdf[traincols].fillna(0).loc[cdf['time'].isin(dates[train])].values
Yt = cdf[targetcols].fillna(0).loc[cdf['time'].isin(dates[train])].values
### validation data
Xv = cdf[traincols].fillna(0).loc[cdf['time'].isin(dates[val])].values
Yv = cdf[targetcols].fillna(0).loc[cdf['time'].isin(dates[val])].values

print("---------------------------> train set built from test set:", Xt.shape, Xv.shape)
print("---------------------------> train model from new train-set:", Xt.shape, Xv.shape)
lgtrain, lgval = lgb.Dataset(Xt, Yt[:,0]), lgb.Dataset(Xv, Yv[:,0])
lgbmodel = lgb.train(params, lgtrain, 2000, valid_sets=[lgtrain, lgval], early_stopping_rounds=300, verbose_eval=200)





# begin prediction

In [ ]:
######## 新的prediction code，由于env.get_prediction_days()只能调用一次，因此改成前面全读进来，在此对数据进行使用
print("============> generating predictions...")
try:
    del cdf
except:
    print('rerun! cdf has been deleted')

import gc
del lgtrain, lgval
gc.collect()
print('read test data from txt file...')
fr = open('market_obs_df.txt','rb')  
market_obs_df = pickle.load(fr)  
fr.close()
fr = open('news_obs_df.txt','rb')  
news_obs_df = pickle.load(fr)  
fr.close()
fr = open('pred_df.txt','rb')  
pred_df = pickle.load(fr)  
fr.close()

###### 开始进行预测
cdf = prepare_data(market_obs_df, news_obs_df)
del market_obs_df, news_obs_df
Xp = cdf[traincols].fillna(0).values
preds = lgbmodel.predict(Xp, num_iteration=lgbmodel.best_iteration)*2 - 1
predsdf = pd.DataFrame({'ast':cdf['assetCode'],'conf':post_scaling(preds)})
predtemplatedf['confidenceValue'][predtemplatedf['assetCode'].isin(predsdf.ast)] = predsdf['conf'].values
predtemplatedf.head(n=5)
predtemplatedf[['time', 'assetCode', 'confidenceValue']].to_csv('self_buld_submission.csv', index=False, float_format='%.8f')


In [ ]:

# ########## 原来的prediction code，按照天来预测，一天天的预测
# print("============> generating predictions...")
# try:
#     del cdf
# except:
#     print('cdf has been deleted')

# preddays = env.get_prediction_days()
# th_ = 0
# for marketdf, newsdf, predtemplatedf in preddays:
#     # print('------- |',th_,'day')
#     cdf = prepare_data(marketdf, newsdf)
#     Xp = cdf[traincols].fillna(0).values
#     preds = lgbmodel.predict(Xp, num_iteration=lgbmodel.best_iteration)*2 - 1
#     # predsdf = pd.DataFrame({'ast':cdf['assetCode'],'conf':post_scaling(preds)})       # 原始程序中，对preds进行了中心化
#     predsdf = pd.DataFrame({'ast':cdf['assetCode'],'conf':post_scaling(preds)})
#     predtemplatedf['confidenceValue'][predtemplatedf['assetCode'].isin(predsdf.ast)] = predsdf['conf'].values
#     env.predict(predtemplatedf)
#     th_ += 1 

# ########## 
# print("============> writing predictions...")
# env.write_submission_file()
# print("============> all finished ...")